In [23]:
from eppy import modeleditor
from eppy.modeleditor import IDF
from matplotlib import pyplot as plt
import os
import pandas as pd
import numpy as np
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem
import matplotlib.pyplot as plt
output_path="./output/simulation_hourly_3years/"

In [24]:
iddfile='./energyplus/EnergyPlus-9-0-1/Energy+.idd'
fname = './input/on_double.idf'
weather='./input/ITA_Torino_160590_IWEC.epw'
IDF.setiddname(iddfile)
idf = IDF(fname,weather)

In [25]:
zones = idf.idfobjects["ZONE"]
total_area=0
for i in range(len(zones)):
    zone = zones[i]
    area = modeleditor.zonearea(idf, zone.Name)
    total_area=total_area+area

print("total one area = %s" % (total_area, ))

total one area = 1084.534275140951


In [26]:
samples_temp =[]
samples_temp.append({'Orientation': 270,
                     'Insulation Thickness': 0.35,
                     'Window to Wall Ratio': 0.15,})
samples = pd.DataFrame.from_dict(samples_temp)
samples

,Orientation,Insulation Thickness,Window to Wall Ratio
0,270,0.35,0.15


In [27]:
building = ef.get_building(fname)
insulation = FieldSelector(class_name='Material', object_name='MW Glass Wool (rolls)_O.1445', 
                           field_name='Thickness')
insulationPR=Parameter(selector=insulation,name='Insulation Thickness')

window_to_wall = wwr(CategoryParameter([0.15])) 

orientation = FieldSelector(class_name='Building', field_name='North Axis')
orientationPR = Parameter(selector=orientation, value_descriptor=CategoryParameter(options=[270]),
                          name='Orientation')

parameters = [orientationPR , window_to_wall, insulationPR]
objectives = ['Electricity:Facility','DistrictHeating:Facility','DistrictCooling:Facility'] # these get made into `MeterReader` or `VariableReader`
problem=EPProblem(parameters, objectives) # problem = parameters + objectives

In [28]:
evaluator = EvaluatorEP(problem, building, out_dir=output_path, err_dir=output_path, epw=weather) # evaluator = problem + building

In [29]:
#hourly simulation
for i in range(len(idf.idfobjects['OUTPUT:VARIABLE'])):
    idf.idfobjects['OUTPUT:VARIABLE'][i].Reporting_Frequency='hourly'
for i in range(len(idf.idfobjects['OUTPUT:METER'])):
    idf.idfobjects['OUTPUT:METER'][i].Reporting_Frequency='hourly'
idf.idfobjects['OUTPUT:ENVIRONMENTALIMPACTFACTORS'][0].Reporting_Frequency='hourly'

In [30]:
#Now we run the evaluator with the given parameters
result = evaluator([270,0.15,0.35]) 
values = dict(zip(objectives, result))
for key, value in values.items():
    print(key, " :: ", "{0:.2f}".format(value/3.6e6), "kWh")

Electricity:Facility  ::  50169.43 kWh
DistrictHeating:Facility  ::  37120.58 kWh
DistrictCooling:Facility  ::  7722.42 kWh


In [31]:
#simulation 2019
run_period = idf.idfobjects['RunPeriod'][0]
run_period.Begin_Year = 2017
run_period.End_Year = 2019
run_period
idf.idfobjects['RunPeriod'][0]
idf.run(readvars=True,output_directory=output_path,annual= True)
idf_data =pd.read_csv(output_path+'eplusout.csv')


/home/ict4bd_v2/Desktop/ICT-for-Building-Design-main/energyplus/EnergyPlus-9-0-1/energyplus --weather /home/ict4bd_v2/Desktop/ICT-for-Building-Design-main/input/ITA_Torino_160590_IWEC.epw --output-directory /home/ict4bd_v2/Desktop/ICT-for-Building-Design-main/output/simulation_hourly_3years --annual --idd ./energyplus/EnergyPlus-9-0-1/Energy+.idd --readvars /home/ict4bd_v2/Desktop/ICT-for-Building-Design-main/in.idf



In [32]:
#getting required columns 
columns=['Date/Time']
for i in idf_data.columns:
    if ('Zone Operative Temperature' in i or 'District' in i or 'Electricity:Facility' in i or 'Drybulb' in i) and ('Hourly' in i): 
        columns=columns+[i]
columns

['Date/Time',
 'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
 'BLOCK1:ZONE3:Zone Operative Temperature [C](Hourly)',
 'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)',
 'BLOCK1:ZONE4:Zone Operative Temperature [C](Hourly)',
 'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)',
 'Electricity:Facility [J](Hourly)',
 'DistrictCooling:Facility [J](Hourly)',
 'DistrictHeating:Facility [J](Hourly)']

In [33]:
df_columns = {'Date/Time':'Date_Time',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)':'t_out',
              'BLOCK1:ZONE3:Zone Operative Temperature [C](Hourly)':'t_in_ZONE3',
              'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)':'t_in_ZONE1',
              'BLOCK1:ZONE4:Zone Operative Temperature [C](Hourly)':'t_in_ZONE4',
              'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)':'t_in_ZONE2',
              'DistrictHeating:Facility [J](Hourly)':'power_heating',
              'DistrictCooling:Facility [J](Hourly)':'power_cooling' ,
              'Electricity:Facility [J](Hourly)':'power_electricity'}


In [34]:
idf_data=idf_data[columns]
data = idf_data.rename(columns =df_columns)
data

,Date_Time,t_out,t_in_ZONE3,t_in_ZONE1,t_in_ZONE4,t_in_ZONE2,power_electricity,power_cooling,power_heating
0,01/01 01:00:00,0.925,13.881985,14.704189,13.783859,13.845197,2.198661e+06,0.0,0.000000e+00
1,01/01 02:00:00,0.650,13.407667,13.970457,13.264381,13.349967,2.198661e+06,0.0,0.000000e+00
2,01/01 03:00:00,0.850,13.093317,13.557124,12.962577,13.026059,2.198661e+06,0.0,7.990230e+05
3,01/01 04:00:00,0.975,12.840838,13.204021,12.814173,12.830734,2.198661e+06,0.0,4.177752e+06
4,01/01 05:00:00,1.000,12.716023,12.894248,12.702670,12.713656,2.198661e+06,0.0,7.533797e+06
...,...,...,...,...,...,...,...,...,...
26275,12/31 20:00:00,0.875,16.786543,16.590183,16.868302,16.838434,4.147716e+07,0.0,8.934283e+07
26276,12/31 21:00:00,0.575,16.853741,16.805404,16.932544,16.905977,4.147716e+07,0.0,8.663718e+07
26277,12/31 22:00:00,0.275,16.903341,16.942140,16.978954,16.955608,4.147716e+07,0.0,8.420745e+07
26278,12/31 23:00:00,0.200,16.938986,17.035979,17.011243,16.990716,4.147716e+07,0.0,8.011249e+07


In [35]:
data['t_in'] = data[['t_in_ZONE3','t_in_ZONE1','t_in_ZONE4', 't_in_ZONE2']].mean(axis=1)

In [36]:
data=data.drop(['t_in_ZONE3', 't_in_ZONE1', 't_in_ZONE4','t_in_ZONE2'], axis = 1)
data

,Date_Time,t_out,power_electricity,power_cooling,power_heating,t_in
0,01/01 01:00:00,0.925,2.198661e+06,0.0,0.000000e+00,14.053807
1,01/01 02:00:00,0.650,2.198661e+06,0.0,0.000000e+00,13.498118
2,01/01 03:00:00,0.850,2.198661e+06,0.0,7.990230e+05,13.159769
3,01/01 04:00:00,0.975,2.198661e+06,0.0,4.177752e+06,12.922441
4,01/01 05:00:00,1.000,2.198661e+06,0.0,7.533797e+06,12.756649
...,...,...,...,...,...,...
26275,12/31 20:00:00,0.875,4.147716e+07,0.0,8.934283e+07,16.770866
26276,12/31 21:00:00,0.575,4.147716e+07,0.0,8.663718e+07,16.874417
26277,12/31 22:00:00,0.275,4.147716e+07,0.0,8.420745e+07,16.945011
26278,12/31 23:00:00,0.200,4.147716e+07,0.0,8.011249e+07,16.994231


In [37]:
data['temp_diff'] =data['t_in'] - data['t_out']

first_year = data[0:8760]
first_year['Date_Time'] = '2017/' + first_year['Date_Time'].str.strip()
second_year = data[8760:17520]
second_year['Date_Time'] = '2018/' + second_year['Date_Time'].str.strip()
third_year = data[17520:]
third_year['Date_Time'] = '2019/' + third_year['Date_Time'].str.strip()

data['Date_Time'] = data['Date_Time'].str.replace('24:00:00','23:59:59')
data

,Date_Time,t_out,power_electricity,power_cooling,power_heating,t_in,temp_diff
0,2017/01/01 01:00:00,0.925,2.198661e+06,0.0,0.000000e+00,14.053807,13.128807
1,2017/01/01 02:00:00,0.650,2.198661e+06,0.0,0.000000e+00,13.498118,12.848118
2,2017/01/01 03:00:00,0.850,2.198661e+06,0.0,7.990230e+05,13.159769,12.309769
3,2017/01/01 04:00:00,0.975,2.198661e+06,0.0,4.177752e+06,12.922441,11.947441
4,2017/01/01 05:00:00,1.000,2.198661e+06,0.0,7.533797e+06,12.756649,11.756649
...,...,...,...,...,...,...,...
26275,2019/12/31 20:00:00,0.875,4.147716e+07,0.0,8.934283e+07,16.770866,15.895866
26276,2019/12/31 21:00:00,0.575,4.147716e+07,0.0,8.663718e+07,16.874417,16.299417
26277,2019/12/31 22:00:00,0.275,4.147716e+07,0.0,8.420745e+07,16.945011,16.670011
26278,2019/12/31 23:00:00,0.200,4.147716e+07,0.0,8.011249e+07,16.994231,16.794231


In [38]:
#converting from J to kWh
data['power_heating'] /= 3.6e6
data['power_cooling'] /= 3.6e6
data['power_electricity'] /= 3.6e6

In [39]:
data

,Date_Time,t_out,power_electricity,power_cooling,power_heating,t_in,temp_diff
0,2017/01/01 01:00:00,0.925,0.610739,0.0,0.000000,14.053807,13.128807
1,2017/01/01 02:00:00,0.650,0.610739,0.0,0.000000,13.498118,12.848118
2,2017/01/01 03:00:00,0.850,0.610739,0.0,0.221951,13.159769,12.309769
3,2017/01/01 04:00:00,0.975,0.610739,0.0,1.160487,12.922441,11.947441
4,2017/01/01 05:00:00,1.000,0.610739,0.0,2.092721,12.756649,11.756649
...,...,...,...,...,...,...,...
26275,2019/12/31 20:00:00,0.875,11.521435,0.0,24.817454,16.770866,15.895866
26276,2019/12/31 21:00:00,0.575,11.521435,0.0,24.065884,16.874417,16.299417
26277,2019/12/31 22:00:00,0.275,11.521435,0.0,23.390958,16.945011,16.670011
26278,2019/12/31 23:00:00,0.200,11.521435,0.0,22.253470,16.994231,16.794231


In [40]:
data['total_power'] = data['power_heating']+ data['power_cooling']
data = data[['Date_Time','t_in','t_out','temp_diff','power_heating','power_cooling','power_electricity','total_power']]

In [41]:
data.to_csv(path_or_buf=output_path+'simulation_data_3years_hourly.csv',index=False)

In [42]:
data

,Date_Time,t_in,t_out,temp_diff,power_heating,power_cooling,power_electricity,total_power
0,2017/01/01 01:00:00,14.053807,0.925,13.128807,0.000000,0.0,0.610739,0.000000
1,2017/01/01 02:00:00,13.498118,0.650,12.848118,0.000000,0.0,0.610739,0.000000
2,2017/01/01 03:00:00,13.159769,0.850,12.309769,0.221951,0.0,0.610739,0.221951
3,2017/01/01 04:00:00,12.922441,0.975,11.947441,1.160487,0.0,0.610739,1.160487
4,2017/01/01 05:00:00,12.756649,1.000,11.756649,2.092721,0.0,0.610739,2.092721
...,...,...,...,...,...,...,...,...
26275,2019/12/31 20:00:00,16.770866,0.875,15.895866,24.817454,0.0,11.521435,24.817454
26276,2019/12/31 21:00:00,16.874417,0.575,16.299417,24.065884,0.0,11.521435,24.065884
26277,2019/12/31 22:00:00,16.945011,0.275,16.670011,23.390958,0.0,11.521435,23.390958
26278,2019/12/31 23:00:00,16.994231,0.200,16.794231,22.253470,0.0,11.521435,22.253470
